In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import csv
import scipy.optimize
import glob
import sys
import traceback

In [ ]:
def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = np.array(tt)
    yy = np.array(yy)
    ff = np.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(np.fft.fft(yy))
    guess_freq = abs(ff[np.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = np.std(yy) * 2.**0.5
    guess_offset = np.mean(yy)
    guess = np.array([guess_amp, 2.*np.pi*guess_freq, 0., guess_offset])

    sinfunc = lambda t, A, w, p, c : A * np.sin(w*t + p) + c
    
    try:
        popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    except Exception as e:
        print(e, "exception in optimizer")
        return {"amp": 0, "omega": 0, "phase": 0, "offset": 0, "freq": f, "period": 1/60, "maxcov": 0,  "pcov": [[0, 0, 0, 0]]}
    
    A, w, p, c = popt
    f = w/(2.*np.pi)
    fitfunc = lambda t: A * np.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": (1./f)/60, "fitfunc": fitfunc, "maxcov": np.max(pcov), "rawres": (guess,popt,pcov), "pcov": pcov}

In [ ]:
curveLength = 240

tt = np.zeros(curveLength, dtype=float)
yy = np.zeros(curveLength, dtype=float)
dataRow = np.zeros(10)

fieldnames = ['amp','freq','phase','offset','maxcov','period','std_dev','nonzeros','amp_err','freq_err']



# x_sample = np.zeros((240)).astype("float32")
# for i in range(0, 239):
#     x_sample[i] = i
x_sample = np.linspace(0, 239, 239, dtype=np.float32)
tt=x_sample
y_full = np.zeros(720)
y_sample = np.zeros((1,240), dtype=np.float32)


#fileDate = "2020-02-06_T2" # contains LSTID

# get list of csv files in this directory
inputDirectory = "E:\detected_edge\\*.csv"

# fileList = []
fileList = glob.glob(inputDirectory)

print("#files:", len(fileList))
print("Enter start date (YYYY-MM-DD), or return to start at top")
startPoint = input()
skip = True if len(startPoint) > 0 else False
    
ft = open("E:\\analyzed_edge\\totals.txt", "w", newline='')
for inputFile in fileList:
    print(inputFile)
    # SCORING VALUES
    ampScore = 0
    covScore = 0
    periodScore = 0
    LSTID_active = False # this is True if current window has LSTID signature
    LSTID_current = False # this is True during LSTID period
    LSTID_start_time = 0
    LSTID_end_time = 0
    fileDate = inputFile[31:41]
    fileDateT = inputFile[31:44]
    if skip == True and fileDate != startPoint:
        print("file date:'"+fileDate+"' startPoint:'"+startPoint+"'")
        continue
    if skip == True and fileDate == startPoint:
        skip = False
    print("open file:",inputFile)

    src_path = "E:\\detected_edge\\detected_edge_" + fileDateT + ".csv")
    dst_path = "E:\\analyzed_edge\\edge_analysis_" + fileDate + ".csv")
    detected_df = pd.read_csv(src_path)
    edge_df = pd.DataFrame()
    
#     with open("E:\\detected_edge\\detected_edge_" + fileDateT + ".csv", newline='') as f:
#         fo = open("E:\\analyzed_edge\\edge_analysis_" + fileDate + ".csv","w", newline='')
    
#     ftotals = csv.writer(ft, delimiter = ",")
#     fwriter = csv.writer(fo, delimiter = ',')
#     fwriter.writerow(fieldnames)
#     reader = csv.reader(f)
    counter = 0
    for row in reader:
        floata = np.asarray(row, dtype=float)
        y_full[counter] = floata[0]
        counter += 1

# iterate thru data by 10 minute increments; in each case, looking at a window of 240 minutes
    for frameStart in range(0, 480, 10):
        for counter, i in enumerate(range(frameStart, frameStart + 240)):
            y_sample[0][counter] = y_full[i]

    yy = y_sample[:1]
    try:
        res = fit_sin(tt, yy[0])
     #   print("start minute:",frameStart)
        print( "Amplitude=%(amp)s, Angular freq.=%(omega)s, phase=%(phase)s, offset=%(offset)s, Max. Cov.=%(maxcov)s" % res )
     #   print("Period=%(period)s hours" % res, " minutes:",res['period'] * 60)
     #   print("%(amp)s, %(omega)s, %(phase)s, %(offset)s, %(maxcov)s, %(period)s" % res)
        tt2 = np.linspace(0, curveLength, 10*curveLength)
        #print("**********res values=", res[0],res[1])
     #   plt.plot(tt, yy[0], "-k", label="y", linewidth=2)

      #  plt.plot(tt2, res["fitfunc"](tt2), "r-", label="y fit curve", linewidth=2)
      #  plt.legend(loc="best")
      #  plt.show()
        
        dataRow[0] = res['amp']
        dataRow[1] = res['omega']
        dataRow[2] = res['phase']
        dataRow[3] = res['offset']
        dataRow[4] = res['maxcov']
        dataRow[5] = res['period']
        dataRow[6] = np.std(yy[0])  # standard deviation
        dataRow[7] = np.count_nonzero(yy[0]) # number of non zeros values
     #   print("pcov= %(pcov)s " % res)
        perr = np.sqrt(np.diag(res.get("pcov")))
    #   print("One Standard deviation errors on parameters=",perr)
        dataRow[8] = perr[0]    # std dev errors on amplitude
        dataRow[9] = perr[1]    # std dev errors on freq
        fwriter.writerow(dataRow)
        if res['amp'] > 3 and res['amp'] < 20:
            ampScore += 1
        if res['maxcov'] < 0.2:
            covScore += 1
        if res['period'] < 4 and res['period'] >= 1:
            periodScore += 1
        # determine if there is an active LSTID period
        # LSTID_active indicates if a period of activity has started
        # LSTID_current indicates if there is LSTID signature in current window
        if abs(res['amp']) > 3 and abs( res['amp']) < 20 and res['maxcov'] < 0.2 \
           and res['period'] < 4 and res['period'] >= 1:  # LSTID signature present
            LSTID_current = True   # signature present in this window
            print(" ")
            print("*************************************")
            print(" ")
        else:
            LSTID_current = False  # signature not present in this window
        if LSTID_current == True:   # LSTID signature present in this window
            if LSTID_active == False:   # activity just started
                LSTID_active == True
                if LSTID_start_time == 0: # first LSTID acivity today
                    LSTID_start_time = i
        else:   # no LSTID signature in this window
            LSTID_current = False      # indicate we are now in period of inactivity
            LSTID_end_time = i        # take note of when activity stopped      
            
    except Exception as e:
        print("exception in fit_sin area")
        print(traceback.format_exc())
        if hasattr(e, 'message'):
          print(e.message)
        else:
          print(e)
        #exit()
  totalScore = ampScore + covScore + periodScore
  print("Date:",fileDate," LSTID score=", totalScore)
  ftotals.writerow([fileDate, totalScore, LSTID_start_time, LSTID_end_time])
  fo.close()
  
  #print("continue?")
# z=input()

ft.close()